In [ ]:
#Import libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import sin, cos, sqrt, atan2, radians

#from dateutil.parser import parse
import datetime as dt

* Unnamed: 0: Index
* Trip_Id: 
* User_Id:
* Sex:
* Birth_year:
* Trip_start:
* Trip_end:
* Origin_Id:
* Destination_Id:
* Age:
* Duration:



In [ ]:
#Import the main dataset
df_bike = pd.read_csv('/kaggle/input/over-9-years-of-real-public-bike-use-data-mibici/mibici_2014-2024/mibici_2014-2024.csv')
display(df_bike)

In [ ]:
#Filter dataset for only trips from a specific month
df_bike = df_bike[df_bike['Trip_start'].str.contains('2023-12-')]
df_bike.head(3)

In [ ]:
#Dataset dimension
df_bike.shape

In [ ]:
#Import the secondary dataset
df_nomenclature = pd.read_csv('/kaggle/input/over-9-years-of-real-public-bike-use-data-mibici/nomenclature_2024.csv')
display(df_nomenclature)

In [ ]:
#Rename columns from the original datasets
df_bike = df_bike.rename(columns={'Unnamed: 0':'Id'})
df_nomenclature = df_nomenclature.rename(columns={'id':'Id','obcn':'Station_code'})

In [ ]:
#Create a new dataset for origin data
df_origin = df_nomenclature[['Id','Station_code','latitude','longitude']]
#Rename columns
df_origin = df_origin.rename(columns={'Id':'Origin_Id','Station_code':'Station_code_origin','latitude':'latitude_origin','longitude':'longitude_origin'})
df_origin.head(3)

In [ ]:
#Create a new dataset for destination data
df_destination = df_nomenclature[['Id','Station_code','latitude','longitude']]
#Rename columns
df_destination = df_destination.rename(columns={'Id':'Destination_Id','Station_code':'Station_code_destination','latitude':'latitude_destination','longitude':'longitude_destination'})
df_destination.head(3)

In [ ]:
#Merge the original dataset with the origin dataset
df = pd.merge(df_bike, df_origin, on='Origin_Id')
#And the destination dataset
df = pd.merge(df, df_destination, on='Destination_Id')
df.head(3)

In [ ]:
#Function to calculate distance using longitude and altitude values
def function_haversine(lat1,lon1,lat2,lon2):
    
    #Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    #Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6373.0 #Approximate radius of earth in km
    return c * r
    
df['Distance'] = function_haversine(df['latitude_origin'], df['longitude_origin'], df['latitude_destination'], df['longitude_destination'])
df.head(3)

In [ ]:
def split_duration(Duration, type):
    
    if type == 'over_1_day':
        time = pd.Series(Duration.split(' ')[0])
        day = time.str.split(' ')[0][0]
        return int(int(day)>0)
    
    elif type == 'over_1_hour':
        time = pd.Series(Duration.split(' ')[2])
        hour = time.str.split(':')[0][0]
        return int(int(hour)>0)
    
    elif type in ['over_45_min','over_30_min','over_15_min','over_5_min','under_5_min']:
        time = pd.Series(Duration.split(' ')[2])
        minute = time.str.split(':')[0][1]
        
        if type == 'over_45_min':
            return int(int(minute)>45)
        elif type == 'over_30_min':
            return int(int(minute)>30)
        elif type == 'over_15_min':
            return int(int(minute)>15)
        elif type == 'over_5_min':
            return int(int(minute)>5)
        else:
            return int(int(minute)<5)

In [ ]:
#SettingWithCopyWarning
pd.options.mode.copy_on_write = True

def split_trip_start(Trip_start, type):
    date = pd.Series(Trip_start.split(' ')[0])
    
    if type == 'Month':
        month = date.str.split('-')[0][1]
        return month
    elif type == 'Year':
        year = date.str.split('-')[0][0]
        return year
    elif type == 'Day':
        day = date.str.split('-')[0][2]
        return day

In [ ]:
df['Duration'].head(5)

In [ ]:
pd.options.mode.copy_on_write = True
def split_time(Duration):
    
    day = int(pd.Series(Duration.split(' ')[0]))
    time = pd.Series(Duration.split(' ')[2])
    hour = int(time.str.split(':')[0][0])
    minute = int(time.str.split(':')[0][1])
    if day > 0:
        return 'over 1 day'
    elif hour > 1:
        return 'over 1 hour'
    elif minute > 45:
        return 'over 45 min'
    elif minute > 30:
        return 'over 30 min'
    elif minute > 15:
        return 'over 15 min'
    elif minute > 5:
        return 'over 5 min'
    elif minute <= 5:
        return 'under 5 min'
    
df['Time'] = df.apply(lambda x: split_time(x['Duration']), axis=1)
df['Time'] 

In [ ]:
df['over_1_hour'] = df.apply(lambda x: split_duration(x['Duration'], 'over_1_hour'), axis=1)
df['over_45_min'] = df.apply(lambda x: split_duration(x['Duration'], 'over_45_min'), axis=1)
df['over_30_min'] = df.apply(lambda x: split_duration(x['Duration'], 'over_30_min'), axis=1)
df['over_15_min'] = df.apply(lambda x: split_duration(x['Duration'], 'over_15_min'), axis=1)
df['over_5_min'] = df.apply(lambda x: split_duration(x['Duration'], 'over_5_min'), axis=1)
df['under_5_min'] = df.apply(lambda x: split_duration(x['Duration'], 'under_5_min'), axis=1)
df['over_1_day'] = df.apply(lambda x: split_duration(x['Duration'], 'over_1_day'), axis=1)

#df['Month'] = df.apply(lambda x: split_trip_start(x['Trip_start'], 'Month'), axis=1)
#df['Year'] = df.apply(lambda x: split_trip_start(x['Trip_start'], 'Year'), axis=1)


In [ ]:
df['Day'] = df.apply(lambda x: split_trip_start(x['Trip_start'], 'Day'), axis=1)
df['formatted_date'] = pd.to_datetime(df['Trip_start'])
df['Week'] = df.formatted_date.apply(lambda x: x.weekofyear)



In [ ]:
df.head(4)

In [ ]:
df2 = df[['Age','Sex','Day','Week','Distance','Time','over_1_hour','over_45_min','over_30_min','over_15_min','over_5_min','under_5_min','Station_code_origin','Station_code_destination']]
df2.head(4)

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
transformer = MinMaxScaler().fit(df[['Distance']])
df[['Min_Max_Distance']] = transformer.transform(df[['Distance']])
df['Min_Max_Distance']

In [ ]:
df_code_station = df.groupby(['User_Id','Sex','Time']).agg({'Trip_Id':'count','Min_Max_Distance':'sum','Station_code_origin':'max'}).sort_values(by='Trip_Id', ascending=False).reset_index()

In [ ]:
df_GDL049 = df_code_station[(df_code_station['Station_code_origin'].str.contains('GDL-049'))]
df_GDL049.head(4)

In [ ]:
df_GDL173 = df_code_station[(df_code_station['Station_code_origin'].str.contains('GDL-173'))]
df_GDL173.head(4)

In [ ]:
fig = plt.figure(figsize=[9,5], facecolor='black', layout='constrained')
fig.suptitle('Station Code Origin', color='white', fontweight='bold', fontsize=16)
fig1, fig2 = fig.subfigures(2, 1) #Create 2 subfigures

fig1.set_facecolor('lightblue')
fig1.suptitle('\nStation Code GDL-049', color='#000040', fontweight='bold') #Subfigure title
ax1 = fig1.add_subplot()
ax1.bar('Time','Trip_Id', color='#000040', data=df_GDL049)
ax1.set_xlabel('Time', fontweight='bold', color='#000040')
ax1.set_ylabel('Number of Trip', fontweight='bold', color='#000040')
ax1.set_xlim(-1,6)
ax1.set_ylim(0,40)
ax1.grid()

fig2.set_facecolor('lightblue')
fig2.suptitle('\nStation Code GDL-173', color='#000040', fontweight='bold') #Subfigure title
ax2 = fig2.add_subplot()
ax2.bar('Time','Trip_Id', color='#000040', data=df_GDL173)
ax2.set_xlabel('Time', fontweight='bold', color='#000040')
ax2.set_ylabel('Number of Trip', fontweight='bold', color='#000040')
ax2.set_xlim(-1,6)
ax2.set_ylim(0, 40)
ax2.grid()

plt.show()





In [ ]:
df3 = df2
df3 = df3.groupby(['week','Sex','Time']).agg({'Distance':'mean','over_1_hour':'count','over_30_min':'count',
                                      'over_15_min':'count','over_5_min':'count','under_5_min':'count',
                                      'Age':'mean'}).reset_index()
df3